In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
from tqdm import tqdm
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import polars as pl
import networkx as nx
from tqdm import tqdm
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/combine-dataset/__results__.html
/kaggle/input/combine-dataset/combine_data.h5
/kaggle/input/combine-dataset/__notebook__.ipynb
/kaggle/input/combine-dataset/__output__.json
/kaggle/input/combine-dataset/custom.css


In [2]:
%%capture
!pip install torch-geometric

# Read data

In [3]:
combine_data=pd.read_hdf("/kaggle/input/combine-dataset/combine_data.h5",key='df')

In [4]:
def display_df(df):
    display(df.shape,df.head())

In [5]:
def get_data(df,entity):
    cols=df.filter(regex=f'^({entity})').columns.to_list()
    data_df=df[cols].explode(cols).drop_duplicates().reset_index(drop=True)
    return data_df
def get_rel(df,col_name):
    rel_df=df[["course_id",col_name]].dropna().explode(col_name).drop_duplicates().reset_index(drop=True)
    return rel_df
    

In [6]:
combine_data.head()

,user_id,course_id,enroll_time,course_name,course_prerequisites,course_about,course_field,school_id,school_name,school_about,...,exercise_num_problem,doing_exercise_id,doing_num_correct,concept_id,concept_prerequisite,concept_name,concept_field,course_total_comments,user_course_num_comment,user_course_num_replies
0,U_24,C_597314,1560360127,university chinese—analysis of northern song d...,,"at present, the talent gap in data science and...",[chinese language and literature],"[S_1, S_14]","[tsinghua university, southeast university]",[referred tsinghua short directly ministry edu...,...,"[1, 6, 4, 2, 4, 3, 2, 6, 3, 4, 5, 6, 4, 2, 2, ...",NaN,NaN,"[K_D_中国语言文学, K_爱国豪情_中国语言文学, K_爱国将领_中国语言文学, K_爱...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[d, patriotic pride, patriotic general, patrio...","[chinese language and literature, chinese lang...",0,0,0
1,U_24,C_597208,1560784961,data mining: theory and algorithm,,the most interesting theory + the most useful ...,[computer science and technology],[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,...,"[5, 3, 5, 3, 5, 6, 3, 3, 5, 5, 4, 5, 5, 2, 5, ...",NaN,NaN,"[K_0 part_计算机科学与技术, K_2 cities_计算机科学与技术, K_2d ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[part, cities, 2d space, matrix, + 2, 3d infor...","[computer science and technology, computer sci...",0,0,0
2,U_24,C_682129,1570379031,software theory basics,,"this course introduces formal language, automa...",NaN,[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,...,"[4, 5, 5, 6, 6, 6, 7, 7, 6, 6, 6, 5, 7, 7, 6, 5]",NaN,NaN,"[K_ASCII_计算机科学与技术, K_cfl的判定性质_计算机科学与技术, K_CYK算...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[ascii, determination of the nature of cfl, cy...","[computer science and technology, computer sci...",13,0,0
3,U_24,C_674910,1570557064,pattern aesthetics and creation,,[national premium course] this course provides...,NaN,[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,...,"[1, 1, 2, 3, 1, 1, 3, 2, 2, 2, 3]",NaN,NaN,NaN,NaN,NaN,NaN,386,0,0
4,U_24,C_707038,1570971985,organic chemistry,,the world's organic compounds are rich and col...,"[chemical engineering and technology, chemical]",[S_1],[tsinghua university],[referred tsinghua short directly ministry edu...,...,"[1, 6, 8, 1, 19, 8, 11, 7, 8, 9, 9, 1, 16, 5, ...",NaN,NaN,"[K_1,4-加成_化学, K_1-溴戊烷_化学, K_1-丁醇_化学, K_1-丁烯_化学...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[1,4-addition, 1-bromopentane, 1-butanol, 1-bu...","[chemical, chemical, chemical, chemical, chemi...",1876,0,0


In [7]:
course_df=combine_data.filter(regex='^(course|teacher|school|video|exercise|concept)').groupby('course_id').first().reset_index()

# User

In [8]:
user_data = combine_data[['user_id','user_gender']].drop_duplicates().reset_index(drop=True)
display_df(user_data)

(367103, 2)

,user_id,user_gender
0,U_24,1
1,U_54,2
2,U_69,1
3,U_105,1
4,U_112,1


In [9]:
course_data = course_df.filter(regex='^(course)').groupby('course_id').first().reset_index()
display_df(course_data)

(2766, 6)

,course_id,course_name,course_prerequisites,course_about,course_field,course_total_comments
0,C_1017355,introduction to traditional chinese medicine e...,the main subjects are college students who hav...,traditional chinese medicine culture - the tre...,None,39624
1,C_1017419,smart government talk,,this course combines specific cases of traffic...,None,609
2,C_1025064,principles of ideological and political education,,"hello, dear classmate! do you want to know how...",None,696
3,C_1025076,advanced roller skating tutorial,,the true meaning of experiencing roller skatin...,None,2
4,C_1025079,scientific fitness method-silk-winding fitness...,none,silk wrapping skills are the chinese solution ...,None,125


In [10]:
school_data = get_data(course_df,"school")
teacher_data = get_data(course_df,"teacher")
video_data = get_data(course_df,"video")
exercise_data = get_data(course_df,"exercise")
concept_data = get_data(course_df,"concept")

In [11]:
user_course=combine_data[['user_id','course_id','enroll_time']]
import pandas as pd

# Giả sử user_course đã được tạo:
# user_course = combine_data[['user_id', 'course_id', 'enroll_time']]

# Sắp xếp theo user và enroll_time
user_course = user_course.sort_values(['user_id', 'enroll_time'])

# Đánh số thứ tự các khóa học của mỗi user
user_course['rank'] = user_course.groupby('user_id').cumcount(ascending=True)

# Tìm khóa học cuối cùng (có rank lớn nhất) của mỗi user
max_rank = user_course.groupby('user_id')['rank'].max().reset_index().rename(columns={'rank': 'max_rank'})

# Merge để gán max_rank vào bảng user_course
user_course = user_course.merge(max_rank, on='user_id')

# Chia dữ liệu
test_set = user_course[user_course['rank'] == user_course['max_rank']].drop(['rank', 'max_rank'], axis=1)
train_set = user_course[user_course['rank'] < user_course['max_rank']].drop(['rank', 'max_rank'], axis=1)

print(f"Số lượng train: {len(train_set)}, Số lượng test: {len(test_set)}")


Số lượng train: 6917397, Số lượng test: 367103


In [12]:
train_set

,user_id,course_id,enroll_time
0,U_1000129,C_735210,1599328511
1,U_1000129,C_799796,1599328642
2,U_1000129,C_1925903,1599403037
3,U_1000129,C_1992970,1599403537
4,U_1000129,C_680884,1602844869
...,...,...,...
7284494,U_99996,C_922850,1578691240
7284495,U_99996,C_936990,1578691372
7284496,U_99996,C_697407,1578691681
7284497,U_99996,C_696679,1578692110


In [13]:
user_course=train_set[['user_id','course_id']]
course_school=get_rel(course_df,"school_id")
course_teacher=get_rel(course_df,"teacher_id")
course_video=get_rel(course_df,"video_ccid")
course_exercise=get_rel(course_df,"exercise_id")
course_concept=get_rel(course_df,"concept_id")
course_field=get_rel(course_df,"course_field")

In [14]:
print("Số user: ", user_course.user_id.nunique())
print("Số khóa học: ", course_data.course_id.nunique())
print("Số trường học: ", school_data.school_id.nunique())
print("Số giảng viên: ", teacher_data.teacher_id.nunique())
print("Số video: ", video_data.video_ccid.nunique())
print("Số exercise: ", exercise_data.exercise_id.nunique())
print("Số lĩnh vực: ", course_field.course_field.nunique())
print("Số liên kết User_Course", user_course.shape[0])
print("Số liên kết Course_School: ", len(course_school))
print("Số liên kết Course_Teacher: ", len(course_teacher))
print("Số liên kết Course_Field: ", len(course_field))
print("Số liên kết Course_Video: ", len(course_video))
print("Số liên kết Course_Exercise: ", len(course_exercise))

Số user:  367103
Số khóa học:  2766
Số trường học:  410
Số giảng viên:  8909
Số video:  55831
Số exercise:  57303
Số lĩnh vực:  78
Số liên kết User_Course 6917397
Số liên kết Course_School:  2747
Số liên kết Course_Teacher:  10655
Số liên kết Course_Field:  564
Số liên kết Course_Video:  67880
Số liên kết Course_Exercise:  57303


In [15]:
print("Course_id: ",course_data.columns.to_list())
print("School_id: ",school_data.columns.to_list())
print("Teacher_id: ",teacher_data.columns.to_list())
print("Video_id: ",video_data.columns.to_list())
print("Exercise_id: ",exercise_data.columns.to_list())

Course_id:  ['course_id', 'course_name', 'course_prerequisites', 'course_about', 'course_field', 'course_total_comments']
School_id:  ['school_id', 'school_name', 'school_about', 'school_motto']
Teacher_id:  ['teacher_id', 'teacher_name', 'teacher_about', 'teacher_job_title', 'teacher_org_name']
Video_id:  ['video_id', 'video_ccid', 'video_duration']
Exercise_id:  ['exercise_id', 'exercise_num_problem']


In [16]:
from sklearn.preprocessing import LabelEncoder

def create_node_mapping(df, id_col):
    """
    Tạo mapping từ node ID gốc sang chỉ số index.
    
    Args:
        df (pd.DataFrame): DataFrame chứa các ID.
        id_col (str): Tên cột ID cần encode.
        
    Returns:
        dict: Mapping từ ID gốc sang index.
        LabelEncoder: Encoder đã fit để encode/transform sau này nếu cần.
    """
    encoder = LabelEncoder()
    node_ids = encoder.fit_transform(df[id_col].unique())
    node_id_map = dict(zip(encoder.classes_, node_ids))
    return node_id_map, encoder

In [17]:
# Tạo mapping
user_id_map, user_encoder = create_node_mapping(user_course, 'user_id')
course_id_map, course_encoder = create_node_mapping(user_course, 'course_id')
teacher_id_map, teacher_encoder = create_node_mapping(course_teacher, 'teacher_id')
school_id_map, school_encoder = create_node_mapping(course_school, 'school_id')
video_id_map, video_encoder = create_node_mapping(course_video, 'video_ccid')
exercise_id_map, exercise_encoder = create_node_mapping(course_exercise, 'exercise_id')
field_id_map, field_encoder = create_node_mapping(course_field, 'course_field')
concept_id_map, concept_encoder = create_node_mapping(course_concept, 'concept_id')

In [18]:
import torch

def create_edge_index(df, src_col, dst_col, src_map, dst_map):
    """
    Tạo edge_index từ dataframe và mapping.
    
    Args:
        df (pd.DataFrame): Bảng chứa các edge.
        src_col (str): Cột ID source node.
        dst_col (str): Cột ID destination node.
        src_map (dict): Mapping ID → index cho source node.
        dst_map (dict): Mapping ID → index cho destination node.
        
    Returns:
        torch.LongTensor: edge_index dạng (2, num_edges)
    """
    edges = df.copy()
    edges['src_idx'] = edges[src_col].map(src_map)
    edges['dst_idx'] = edges[dst_col].map(dst_map)
    edge_index = torch.tensor([edges['src_idx'].values, edges['dst_idx'].values], dtype=torch.long)
    return edge_index

# Tạo từng edge_index
user_course_edge_index = create_edge_index(user_course, 'user_id', 'course_id', user_id_map, course_id_map)
course_teacher_edge_index = create_edge_index(course_teacher, 'course_id', 'teacher_id', course_id_map, teacher_id_map)
course_school_edge_index = create_edge_index(course_school, 'course_id', 'school_id', course_id_map, school_id_map)
course_video_edge_index = create_edge_index(course_video, 'course_id', 'video_ccid', course_id_map, video_id_map)
course_exercise_edge_index = create_edge_index(course_exercise, 'course_id', 'exercise_id', course_id_map, exercise_id_map)
course_field_edge_index = create_edge_index(course_field, 'course_id', 'course_field', course_id_map, field_id_map)
course_concept_edge_index = create_edge_index(course_concept, 'course_id', 'concept_id', course_id_map, concept_id_map)


<ipython-input-18-a5b1d3c65adb>:20: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  edge_index = torch.tensor([edges['src_idx'].values, edges['dst_idx'].values], dtype=torch.long)


In [19]:
from torch_geometric.data import HeteroData

data = HeteroData()

# Khởi tạo node (số lượng node là bắt buộc)
data['user'].num_nodes = len(user_id_map)
data['course'].num_nodes = len(course_id_map)
data['teacher'].num_nodes = len(teacher_id_map)
data['school'].num_nodes = len(school_id_map)
data['video'].num_nodes = len(video_id_map)
data['exercise'].num_nodes = len(exercise_id_map)
data['field'].num_nodes = len(field_id_map)
data['concept'].num_nodes = len(concept_id_map)

# Ví dụ: Khởi tạo random feature 16 chiều cho mỗi node type
node_dim = 16
data['user'].x = torch.randn(data['user'].num_nodes, node_dim)
data['course'].x = torch.randn(data['course'].num_nodes, node_dim)
data['teacher'].x = torch.randn(data['teacher'].num_nodes, node_dim)
data['school'].x = torch.randn(data['school'].num_nodes, node_dim)
data['video'].x = torch.randn(data['video'].num_nodes, node_dim)
data['exercise'].x = torch.randn(data['exercise'].num_nodes, node_dim)
data['field'].x = torch.randn(data['field'].num_nodes, node_dim)
data['concept'].x = torch.randn(data['concept'].num_nodes, node_dim)

# Thêm edge cho từng loại
data['user', 'enrolls', 'course'].edge_index = user_course_edge_index
data['course', 'taught_by', 'teacher'].edge_index = course_teacher_edge_index
data['course', 'belongs_to', 'school'].edge_index = course_school_edge_index
data['course', 'has_video', 'video'].edge_index = course_video_edge_index
data['course', 'has_exercise', 'exercise'].edge_index = course_exercise_edge_index
data['course', 'has_field', 'field'].edge_index = course_field_edge_index
data['course', 'has_concept', 'concept'].edge_index = course_concept_edge_index


In [20]:
# Đảm bảo tất cả node types đều có feature
for node_type in data.node_types:
    if 'x' not in data[node_type]:
        num_nodes = data[node_type].num_nodes
        data[node_type].x = torch.randn(num_nodes, node_dim)


In [21]:
import torch
from torch_geometric.data import HeteroData
from torch_geometric.nn import SAGEConv, HeteroConv
from torch.nn import Module
from torch.nn.functional import binary_cross_entropy_with_logits
from torch_geometric.transforms import ToUndirected
# 1. Build HeteroGNN model
class HeteroGNN(Module):
    def __init__(self, metadata, hidden_channels=32):
        super().__init__()
        self.conv1 = HeteroConv({
            edge_type: SAGEConv((-1, -1), hidden_channels)
            for edge_type in metadata[1]
        }, aggr='sum')

        self.conv2 = HeteroConv({
            edge_type: SAGEConv((-1, -1), hidden_channels)
            for edge_type in metadata[1]
        }, aggr='sum')

    def forward(self, x_dict, edge_index_dict):
        x_dict = self.conv1(x_dict, edge_index_dict)
        x_dict = {key: x.relu() if x is not None else x_dict[key] for key, x in x_dict.items()}
        x_dict = self.conv2(x_dict, edge_index_dict)
        x_dict = {key: x if x is not None else x_dict[key] for key, x in x_dict.items()}
        return x_dict

In [22]:
# 2. Link prediction decoder
def decode(source_node, target_node):
    return (source_node * target_node).sum(dim=-1)

# 3. Loss computation
def compute_loss(out, edge_index_dict, target_edge_type, device):
    src, dst = edge_index_dict[target_edge_type]
    pos_out = decode(out[target_edge_type[0]][src], out[target_edge_type[2]][dst])

    # Negative sampling
    num_nodes = out[target_edge_type[2]].size(0)
    neg_dst = torch.randint(0, num_nodes, (src.size(0),), device=device)
    neg_out = decode(out[target_edge_type[0]][src], out[target_edge_type[2]][neg_dst])

    pos_loss = binary_cross_entropy_with_logits(pos_out, torch.ones_like(pos_out))
    neg_loss = binary_cross_entropy_with_logits(neg_out, torch.zeros_like(neg_out))

    return pos_loss + neg_loss

In [23]:
# 4. Training function
def train(model, data, optimizer, target_edge_type, epochs=10, device='cpu'):
    model.train()
    for epoch in tqdm(range(1, epochs + 1), desc=f"Training Progress"):
        optimizer.zero_grad()
        out = model(data.x_dict, data.edge_index_dict)
        loss = compute_loss(out, data.edge_index_dict, target_edge_type, device)
        loss.backward()
        optimizer.step()

        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

# 5. Inference (predict score for all edges)
def predict_links(model, data, target_edge_type):
    model.eval()
    with torch.no_grad():
        out = model(data.x_dict, data.edge_index_dict)
        src, dst = data[target_edge_type].edge_index
        scores = torch.sigmoid(decode(out[target_edge_type[0]][src], out[target_edge_type[2]][dst]))
        return scores

In [24]:
for edge in data.metadata()[1]: 
    print(edge)

('user', 'enrolls', 'course')
('course', 'taught_by', 'teacher')
('course', 'belongs_to', 'school')
('course', 'has_video', 'video')
('course', 'has_exercise', 'exercise')
('course', 'has_field', 'field')
('course', 'has_concept', 'concept')


In [25]:

data = ToUndirected()(data)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HeteroGNN(data.metadata()).to(device)
data = data.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
train(model, data, optimizer, target_edge_type=('user', 'enrolls', 'course'), epochs=500, device=device)


Training Progress:   0%|          | 1/500 [00:00<07:18,  1.14it/s]

Epoch 1, Loss: 3.7177


Training Progress:   0%|          | 2/500 [00:01<04:17,  1.93it/s]

Epoch 2, Loss: 3.6329


Training Progress:   1%|          | 3/500 [00:01<03:19,  2.49it/s]

...

Epoch 498, Loss: 1.1684


Training Progress: 100%|█████████▉| 499/500 [02:12<00:00,  3.77it/s]

Epoch 499, Loss: 1.1684


Training Progress: 100%|██████████| 500/500 [02:13<00:00,  3.76it/s]

Epoch 500, Loss: 1.1672


In [26]:
def predict_topk_links_per_src(model, data, target_edge_type, src_list, top_k=5, score_threshold=0.9):
    """
    Predict top-k negative links cho mỗi node src trong src_list.
    Lọc các link có score >= threshold và đảm bảo mỗi src có ít nhất 1 link.

    Args:
        model: Trained HeteroGNN model.
        data: HeteroData object.
        target_edge_type: Tuple, e.g., ('user', 'enrolls', 'course')
        src_list: Danh sách node src cần predict.
        top_k: Số lượng link có score cao nhất muốn lấy cho mỗi src.
        score_threshold: Ngưỡng score cần giữ.

    Returns:
        pandas.DataFrame với 3 cột: [source_name, target_name, score] với src và dst là kiểu int.
    """
    import pandas as pd
    import torch
    from tqdm import tqdm

    model.eval()
    with torch.no_grad():
        out = model(data.x_dict, data.edge_index_dict)

        num_dst_nodes = data[target_edge_type[2]].num_nodes
        existing_edges = set(zip(
            data[target_edge_type].edge_index[0].tolist(),
            data[target_edge_type].edge_index[1].tolist()
        ))

        results = []

        for src in tqdm(src_list, desc="Predicting for each source node"):
            existing_dst = {dst for s, dst in existing_edges if s == src}
            candidate_dst = [dst for dst in range(num_dst_nodes) if dst not in existing_dst]

            if len(candidate_dst) == 0:
                continue

            dst_indices = torch.tensor(candidate_dst, device=next(model.parameters()).device)
            src_indices = torch.full_like(dst_indices, src)

            scores = torch.sigmoid(decode(out[target_edge_type[0]][src_indices], out[target_edge_type[2]][dst_indices]))

            topk_scores, topk_indices = torch.topk(scores, min(top_k, len(scores)))
            topk_dst = dst_indices[topk_indices]

            for s, d, sc in zip(src_indices[topk_indices].cpu().numpy(), topk_dst.cpu().numpy(), topk_scores.cpu().numpy()):
                results.append((s, d, sc))

        # Tạo DataFrame kết quả
        df = pd.DataFrame(results, columns=[target_edge_type[0], target_edge_type[2], 'score'])

        # Lọc theo score_threshold
        filtered_df = df[df['score'] >= score_threshold].copy()

        # Đảm bảo mỗi src còn ít nhất 1 dòng
        all_src = df[target_edge_type[0]].unique()
        remaining_src = filtered_df[target_edge_type[0]].unique()
        missing_src = set(all_src) - set(remaining_src)

        for src in missing_src:
            best_row = df[df[target_edge_type[0]] == src].sort_values(by='score', ascending=False).iloc[0]
            filtered_df = pd.concat([filtered_df, pd.DataFrame([best_row])], ignore_index=True)

        # Ép kiểu src và dst về int
        filtered_df[target_edge_type[0]] = filtered_df[target_edge_type[0]].astype(int)
        filtered_df[target_edge_type[2]] = filtered_df[target_edge_type[2]].astype(int)

        return filtered_df.reset_index(drop=True)


def map_course_ids_to_indices(course_ids, course_id_mapping):
    """
    Chuyển danh sách course_id sang index dựa trên mapping.

    Args:
        course_ids: List các course_id (tên gốc).
        course_id_mapping: Dictionary ánh xạ course_id -> index.

    Returns:
        List các index của course_ids.
    """
    indices = []
    for course_id in course_ids:
        if course_id in course_id_mapping:
            indices.append(course_id_mapping[course_id])
        else:
            print(f"Warning: course_id {course_id} không có trong mapping.")
    return indices


In [27]:
e_types = data.metadata()[1]
e_types

[('user', 'enrolls', 'course'),
 ('course', 'taught_by', 'teacher'),
 ('course', 'belongs_to', 'school'),
 ('course', 'has_video', 'video'),
 ('course', 'has_exercise', 'exercise'),
 ('course', 'has_field', 'field'),
 ('course', 'has_concept', 'concept'),
 ('course', 'rev_enrolls', 'user'),
 ('teacher', 'rev_taught_by', 'course'),
 ('school', 'rev_belongs_to', 'course'),
 ('video', 'rev_has_video', 'course'),
 ('exercise', 'rev_has_exercise', 'course'),
 ('field', 'rev_has_field', 'course'),
 ('concept', 'rev_has_concept', 'course')]

In [28]:
missing_school_course=course_df[course_df["school_id"].isnull()].course_id.to_list()
missing_school_ids=map_course_ids_to_indices(missing_school_course,course_id_map)

In [29]:
missing_teacher_course=course_df[course_df["teacher_id"].isnull()].course_id.to_list()
missing_teacher_ids=map_course_ids_to_indices(missing_teacher_course,course_id_map)

In [30]:
missing_field_course=course_df[course_df["course_field"].isnull()].course_id.to_list()
missing_field_ids=map_course_ids_to_indices(missing_field_course,course_id_map)

In [31]:
topk_school = predict_topk_links_per_src(model, 
                                     data, 
                                     e_types[2], 
                                     src_list=missing_school_ids, 
                                     top_k=2,
                                     score_threshold=0.97
                                        )
topk_school.head()


Predicting for each source node: 100%|██████████| 40/40 [00:00<00:00, 352.38it/s]


,course,school,score
0,1908,98,0.999976
1,1908,400,0.999953
2,1688,251,0.993452
3,1688,296,0.988654
4,2602,231,0.997051


In [32]:
topk_teacher = predict_topk_links_per_src(model, 
                                     data, 
                                     e_types[1], 
                                     src_list=missing_teacher_ids, 
                                     top_k=5,
                                     score_threshold=0.999
                                         )
topk_teacher.head()


Predicting for each source node: 100%|██████████| 16/16 [00:00<00:00, 287.27it/s]


,course,teacher,score
0,1593,4959,1.0
1,1593,3403,1.0
2,1593,6954,1.0
3,1593,6971,1.0
4,1593,677,1.0


In [33]:
topk_field = predict_topk_links_per_src(model, 
                                     data, 
                                     e_types[5], 
                                     src_list=missing_field_ids, 
                                     top_k=5,
                                     score_threshold=0.97
                                       )

topk_field.head()


Predicting for each source node: 100%|██████████| 2317/2317 [00:00<00:00, 2485.67it/s]


,course,field,score
0,2112,29,0.999994
1,2112,0,0.999956
2,2112,31,0.999702
3,2112,23,0.999629
4,2112,28,0.999436


In [34]:
def reverse_dataframe_ids(df, id_mappings,rename_col):
    """
    Chuyển các cột index trong DataFrame về ID gốc dựa trên mapping.

    Args:
        df: DataFrame chứa các cột index.
        id_mappings: Dictionary với key là tên cột, value là mapping ID -> index.

    Returns:
        DataFrame với các cột ID gốc.
    """
    df_converted = df.copy()
    
    for col, mapping in id_mappings.items():
        reverse_mapping = {v: k for k, v in mapping.items()}
        if col in df_converted.columns:
            df_converted[col] = df_converted[col].map(reverse_mapping)
    df_converted.rename(columns={'course': 'course_id', rename_col[0]: rename_col[1]}, inplace=True)
    return df_converted
import pandas as pd

# Ánh xạ ngược
id_mappings = {'user': user_id_map,
               'course': course_id_map,
               'school': school_id_map,
               'teacher':teacher_id_map,
               'field': field_id_map
              }

# Chuyển index về ID gốc



In [35]:
fill_school = reverse_dataframe_ids(topk_school, id_mappings, ["school","school_id"])
fill_teacher = reverse_dataframe_ids(topk_teacher, id_mappings, ["teacher","teacher_id"])
fill_field = reverse_dataframe_ids(topk_field, id_mappings, ["field","course_field"])

In [36]:
course_field.value_counts("course_id").describe()

count    449.000000
mean       1.256125
std        0.569981
min        1.000000
25%        1.000000
50%        1.000000
75%        1.000000
max        7.000000
Name: count, dtype: float64

In [37]:
course_teacher.value_counts("course_id").describe()

count    2750.000000
mean        3.874545
std         3.604881
min         1.000000
25%         1.000000
50%         3.000000
75%         5.000000
max        49.000000
Name: count, dtype: float64

In [38]:
import pandas as pd
import numpy as np

def replace_list_column(df_target, df_fill, key_col, value_col):
    """
    Thay thế giá trị của cột chứa list trong df_target bằng giá trị từ df_fill nếu có.

    Args:
        df_target (pd.DataFrame): DataFrame gốc.
        df_fill (pd.DataFrame): DataFrame chứa dữ liệu mới để thay thế.
        key_col (str): Tên cột khóa.
        value_col (str): Tên cột chứa list cần thay thế.

    Returns:
        pd.DataFrame: DataFrame đã thay thế.
    """
    # Tạo mapping
    mapping = dict(zip(df_fill[key_col], df_fill[value_col]))

    # Hàm thay thế từng dòng
    def replace_row(row):
        if not row[value_col]:
            return mapping[row[key_col]]
        return row[value_col]   # Giữ nguyên nếu không có trong mapping

    # Thay thế
    df_target[value_col] = df_target.apply(replace_row, axis=1)

    return df_target


In [39]:
gr_field=fill_field[['course_id','course_field']].groupby('course_id').agg(list).reset_index()
gr_school=fill_school[['course_id','school_id']].groupby('course_id').agg(list).reset_index()
gr_teacher=fill_teacher[['course_id','teacher_id']].groupby('course_id').agg(list).reset_index()

In [40]:
new_df=replace_list_column(course_df,gr_field,"course_id","course_field")
new_df=replace_list_column(new_df,gr_school,"course_id","school_id")
new_df=replace_list_column(new_df,gr_teacher,"course_id","teacher_id")

In [41]:
def fill_attributes_by_id(course_df, id_col, attr_cols):
    # Merge
    df = course_df[[id_col]+attr_cols].dropna().explode([id_col]+attr_cols)
    df = df.drop_duplicates()
    rel_df = course_df[['course_id',id_col]].explode(id_col)
    df = pd.merge(rel_df,df,on=id_col,how='left')
    df = df.groupby('course_id').agg(list).reset_index()
    course_df = course_df.drop([id_col]+attr_cols,axis=1)
    course_df = pd.merge(course_df,df,on='course_id',how='left')
    return course_df
# Bổ sung thông tin trường học
def fill(course_df,entities):
    for ent in entities:
        cols = course_df.columns[course_df.columns.str.startswith(ent)].tolist()
        attr_cols = [item for item in cols if not (isinstance(item, str) and item.endswith('_id'))]
        id_col = ent+'_id'
        course_df = fill_attributes_by_id(course_df,id_col,attr_cols)
    return course_df
entities=['school','teacher']
new_df = fill(new_df,entities)

In [42]:
new_df.isna().mean()

course_id                0.000000
course_name              0.000000
course_prerequisites     0.000000
course_about             0.000000
course_field             0.000000
video_id                 0.532538
video_ccid               0.532538
video_duration           0.532538
exercise_id              0.022054
exercise_num_problem     0.022054
concept_id               0.732104
concept_prerequisite     0.732104
concept_name             0.732104
concept_field            0.732104
course_total_comments    0.000000
school_id                0.000000
school_name              0.000000
school_about             0.000000
school_motto             0.000000
teacher_id               0.000000
teacher_name             0.000000
teacher_about            0.000000
teacher_job_title        0.000000
teacher_org_name         0.000000
dtype: float64

In [43]:
course_info_cols = new_df.drop(['course_id'],axis=1).columns.tolist()
user_interact = combine_data.drop(course_info_cols,axis=1)

In [44]:
user_course = combine_data[['user_id','course_id']]
del combine_data
combine_data = pd.merge(user_course,new_df,on='course_id',how='left')
combine_data = pd.merge(combine_data,user_interact,on=['user_id','course_id'],how='left')

In [45]:
combine_data.isnull().mean()

user_id                    0.000000
course_id                  0.000000
course_name                0.000000
course_prerequisites       0.000000
course_about               0.000000
course_field               0.000000
video_id                   0.379174
video_ccid                 0.379174
video_duration             0.379174
exercise_id                0.019379
exercise_num_problem       0.019379
concept_id                 0.589986
concept_prerequisite       0.589986
concept_name               0.589986
concept_field              0.589986
course_total_comments      0.000000
school_id                  0.000000
school_name                0.000000
school_about               0.000000
school_motto               0.000000
teacher_id                 0.000000
teacher_name               0.000000
teacher_about              0.000000
teacher_job_title          0.000000
teacher_org_name           0.000000
enroll_time                0.000000
user_gender                0.000000
watch_id                   0

In [46]:
combine_data.to_hdf("data_ver5.h5",key='df')

<ipython-input-46-b69a6c86904d>:1: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->Index(['user_id', 'course_id', 'course_name', 'course_prerequisites',
       'course_about', 'course_field', 'video_id', 'video_ccid',
       'video_duration', 'exercise_id', 'exercise_num_problem', 'concept_id',
       'concept_prerequisite', 'concept_name', 'concept_field', 'school_id',
       'school_name', 'school_about', 'school_motto', 'teacher_id',
       'teacher_name', 'teacher_about', 'teacher_job_title',
       'teacher_org_name', 'watch_id', 'total_watch_time', 'actual_watch_time',
       'doing_exercise_id', 'doing_num_correct'],
      dtype='object')]

  combine_data.to_hdf("data_ver5.h5",key='df')
